# UFSM
## Engenharia Acústica 
## Processamento Digital de Sinais II
### Trabalho Integrado: rotina de medição

In [1]:
# Carregando as bibliotecas que utilizaremos
import pytta
from scipy.io import wavfile as wf
import copy as cp
import numpy as np

#### Organização dos dados medidos
 
O tipo de dado Struct, do MATLAB, funciona de forma semelhante ao Dictionary do Python. Vamos organizar as variáveis com dicionários dentro de dicionários.

Sugiro que os dados sejam organizados nos seguintes níveis de informação e sequência:
1. Configuração fonte receptor; e.g. S1R1;
2. Sinal de excitação: sweep, música ou fala;
3. Sinal gravado: biauricular, monofônico

É notável que no momento da medição os dados não serão adquiridos nessa sequência. Uma parte do trabalho da nossa rotina de medição é pegar os dados medidos na última tomada e inseri-los no lugar correto do nosso dicionário de medição. Pois a cada tomada o número de posições medidas de receptor  mudará, como já vimos na matriz de teste (e.g. tomada 1 com S1R1 - biauricular, S1R2 - centro da cabeça - e S1R3 - centro do cabeça; tomada 'n' somente com S2R5 - biauricular).

#### Exemplo de tomada de medição e armazenamento na matriz de dados

In [2]:
# Load de sinal de áudio qualquer para demonstração do código
fileName = 'Piano Over the rainbow Mic2  SHORT.wav'
dummySignal = pytta.read_wav(fileName)   

# Cria signalObj de 4 canais como seria numa tomada de medição
measureTake = pytta.merge(dummySignal,dummySignal)
measureTake = pytta.merge(measureTake,measureTake) # bug aqui! pytta.merge em adaptação. :x
measureTake.channelName = {1:'S1R1',2:'S1R1',3:'S1R2',4:'S1R3'}
measureTake.comment = 'music'
measureTake.temp = 24.666
measureTake.RH = 69

# Desmembra o SignalObj measuretake de 4 canais em 3 SignalObj referentes ao arranjo biauricular 
# em uma posição e ao centro da cabeça em duas outras posições

binaural = pytta.SignalObj(measureTake.timeSignal[:,0:1],
                           'time',
                           samplingRate=measureTake.samplingRate,
                           channelName={1:measureTake.channelName[1],2:measureTake.channelName[2]},
                           comment=measureTake.comment)
binaural.temp = measureTake.temp
binaural.RH = measureTake.RH

hc1 = pytta.SignalObj(measureTake.timeSignal[:,2],
                      'time',
                      samplingRate=measureTake.samplingRate,
                      channelName={1:measureTake.channelName[3]},
                      comment=measureTake.comment)
hc1.temp = measureTake.temp
hc1.RH = measureTake.RH

hc2 = pytta.SignalObj(measureTake.timeSignal[:,3],
                      'time',
                      samplingRate=measureTake.samplingRate,
                      channelName={1:measureTake.channelName[4]},
                      comment=measureTake.comment)
hc2.temp = measureTake.temp
hc2.RH = measureTake.RH

    
# Neste ponto já teríamos tomado uma medição e desmembrado o SignalObj resultante em 3 novos SignalObj, sendo um 
# para o microfone biauricular e dois para os microfones de centro da cabeça.
    
# Adicionando a última tomada da medição no dicionário de dados
    
measurementData = {} # Cria o dicionário vazio que conterá todos os níveis de informação do nosso dia de medição

# Adiciona cada uma das três posições de receptor da última tomada de medição 
measurementData[binaural.channelName[1]] = {binaural.comment : {'binaural' : binaural }}
measurementData[hc1.channelName[1]] = {hc1.comment : {'hc' : hc1 }}
measurementData[hc2.channelName[1]] = {hc2.comment : {'hc' : hc2 }}




AttributeError: No 'pyttaObj' is able handle arrays with more                         than 2 dimensions, '[:,:]', YET!.

#### Acessando dados do dicionário de medição

In [ ]:
measurementData['S1R1']['music']['binaural'].play()